In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup

import pandas as pd

In [99]:
def move_loop():
    next_page_element = driver.find_element(By.XPATH, f"//a[contains(@onclick, 'procStoremapGetItemList({next_page})')]")
    driver.execute_script("arguments[0].click();", next_page_element)

    return print(f"{next_page} 페이지로 이동했습니다.")

In [104]:
#주소 불러오기
url = 'https://composecoffee.com/findstore'

# 자동화 크롬 도구 옵션 지정
chrome_option = Options()
chrome_option.add_experimental_option('detach', True)
driver = webdriver.Chrome(options=chrome_option)

# 크롤링 시작
driver.get(url)
driver.maximize_window()

wait = WebDriverWait(driver,10)

# ************************************************************************
# ************************************************************************
# ************************처음에는 지역별로 찾을랬는데,***********************
# ************페이지 로딩이 이상한거 같아, 전체에서 페이징으로 찾기로 결정********
# ************************************************************************
# ************************************************************************
# # search_keyword 찾을때 까지 기다리기
# wait.until(
#     EC.presence_of_element_located((By.ID, 'search_keyword'))
# )
# print('찾음')
# # search_keyword 찾으면 지역 입력하고 엔터치기
# find_search = driver.find_element(by = By.ID, value='search_keyword')
# find_search.send_keys('서울')
# find_search.send_keys(Keys.RETURN)
# ************************************************************************
# ************************************************************************

#여기 append 할꺼임
final_list = []

#첨에 변수에 값 줌
current_page = 1
max_page = 10


while current_page <= max_page:

    # 쪽수가 전부 나올때까지 기다리기
    wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#content > div > section.storemap > div.right_info > ul > div'))
    )

    # 찾아야하는 요소 나올때까지 기다리기
    wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#content div.item_right'))
    )

    # 쪽수 나오면 html 파싱하기
    compose_html = driver.page_source
    soup = BeautifulSoup(compose_html, 'html.parser')

    # 나머지 페이지 찾아서 Max 값 찾기
    try :
        max_page = max([int(i.text) for i in soup.select('#content > div > section.storemap > div.right_info > ul > div > a')])
        # print(f'최대 페이지 : {max_page} ')
        pass

    # 마지막 페이지는 timeout 에러 처리해서 마지막 한번 더 추출해주기
    except TimeoutException:
        print('마지막 페이지같은데요!')
        pass


    #지점명 추출
    francise_list = [i.text for i in soup.select('#content div.item_right div.item_title')]

    #주소 추출
    address_list = [i.text for i in soup.select('#content div.item_right div.item_address')]

    # 전화번호 추출(리스트 컴프리헨션으로 구현)
    # TEL_list = []
    # for i in soup.select('#content div.item_right div.item_phone_number'):
    #     if i.text == '':
    #         i = 'NULL'
    #         TEL_list.append(i)
    #     else:
    #         TEL_list.append(i.text)
    TEL_list = [i.text if i.text != '' else 'NULL' for i in soup.select('#content div.item_right div.item_phone_number')]


    for x,y,z in zip(francise_list, address_list, TEL_list):
        # 중복제거를 하고 들어가자
        if [x,y,z] not in final_list:
            final_list.append([x,y,z])
            
        else:
            pass
    
    print(f'''
          현재 페이지 : {current_page}
          찾은 데이터 : {francise_list}
            ''')
    

    # 다음 페이지 번호 계산
    next_page = current_page +1

    try:
        #다음 페이지 클릭까지 대기
        wait.until(
            EC.presence_of_element_located((By.XPATH, f"//a[contains(@onclick, 'procStoremapGetItemList({next_page})')]"))
        )
        # 다음 페이지 링크 클릭
        try:
            next_page_element = driver.find_element(By.XPATH, f"//a[contains(@onclick, 'procStoremapGetItemList({next_page})')]")
            driver.execute_script("arguments[0].click();", next_page_element)

            #클릭 후 다음 크롤링 요소 찾을때까지 대기
            wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.item_right'))
            )

            # 한번 더 wait 해줌
            wait

        except Exception as e:
            break
        
        # 현재 page + 1
        current_page += 1
        pass
    except TimeoutException:
        print('마지막 페이지이네요~!')
        pass





          현재 페이지 : 1
          찾은 데이터 : ['일산만자고개점', '경기광주밀목점', '수원시청역점', '인천청학용담공원점', '양산경동스마트점', '마산합성두산위브점', '장림두산제니스점', '경북청도냉정점', '당정역점', '전남곡성옥과점', '청주가경점', '천안역점', '종로르메이에르점', '수원영통벽적골점', '광주첨단쌍암점', '인천성지초등학교점', '대전중리점', '고양DMC플렉스데시앙점', '마천중앙시장점', '광주운암롯데캐슬점']
            

          현재 페이지 : 2
          찾은 데이터 : ['일산만자고개점', '경기광주밀목점', '수원시청역점', '인천청학용담공원점', '양산경동스마트점', '마산합성두산위브점', '장림두산제니스점', '경북청도냉정점', '당정역점', '전남곡성옥과점', '청주가경점', '천안역점', '종로르메이에르점', '수원영통벽적골점', '광주첨단쌍암점', '인천성지초등학교점', '대전중리점', '고양DMC플렉스데시앙점', '마천중앙시장점', '광주운암롯데캐슬점']
            

          현재 페이지 : 3
          찾은 데이터 : ['상계보람점', '파주동문그린시티점', '광주평동점', '진천터미널점', '과천RF사옥점', '춘천거두리점', '창원성주동점', '용인한숲시티점', '천안자이타워점', '봉담수기교차로점', '회현SK리더스점', '선릉한티중앙점', '수원탑동점', '광주상무엘리체점', '인천검단제일프라자점', '화곡역점', '정선민둥산역점', '양산상북보건소점', '무안오룡푸르지오점', '인천약산초점']
            

          현재 페이지 : 4
          찾은 데이터 : ['광주하남지식센터점', '세종부강점', '서초1동점', '인천연수영남점', '인천청라국제도시역점', '중앙대광명병원점', '용인구성점', '여주현암점', '광주시청역점', '예천시장점', '담양백동주공점', '안양종합운동

# 궁금한점
왜 1페이지와 2페이지의 추출된 데이터가 같을까?

왜일까

첨에 current_page에 0으로 시작했는데 동일하다


In [139]:
address_list

['부산 사하구 낙동대로516번길 48 (하단동, 하단에덴아파트)', '부산 남구 용소로21번길 25 1층']

In [141]:
#데이터 프레임으로 만들기
df = pd.DataFrame(final_list, columns= ['francise_name', 'address', 'TEL'])
df[-2:]

,francise_name,address,TEL
2527,동아대점,"부산 사하구 낙동대로516번길 48 (하단동, 하단에덴아파트)",051-294-2638
2528,부경대점,부산 남구 용소로21번길 25 1층,NULL


### 이상한 특수문자(\u2013) 처리

엔 대시와 하이픈의 차이
하이픈 (-): U+002D

일반적인 텍스트 구분에 사용.
길이가 짧음.
엔 대시 (–): U+2013

범위를 나타내는 데 주로 사용.
길이가 더 길음.
엠 대시 (—): U+2014

문장 부호로 사용.
엔 대시보다 더 길음.

In [199]:
mask_filter = df.apply(lambda col : col.str.contains('\u2013', na = False))
row_number = df[mask_filter.any(axis=1)].index

#상태 확인
print(f'수정 전 : {df[mask_filter.any(axis=1)]}')

# df 전체 하이픈으로 대체해줌(regex는 부분적으로 일치하는 것을 바꿔줌. 기본적으로 replace는 모두 일치해야 바꿈)
df.replace('\u2013', '-', regex = True, inplace= True)

# 바꼈는지 확인
df.loc[row_number]

수정 전 :    francise_name                             address            TEL
80      인천더샵아르테점  인천 미추홀구 구월로8번길 18-62 근생1동 b1층 104호  070–7585-7235


,francise_name,address,TEL
80,인천더샵아르테점,인천 미추홀구 구월로8번길 18-62 근생1동 b1층 104호,070-7585-7235


### 이상한 데이터 처리

#### 확인 결과
html 자체에 전화번호 요소에 주소가 들어가있음

In [227]:
# df.loc[46:46]['address'] = df.loc[46:46]['address'] + df.loc[46:46]['TEL']

# df.loc[46:46]
# df.loc[row_indexer, "col"]

# 원래 주소에 TEL을 합쳐주기
backup_address = df.loc[46, 'address']
df.loc[46, 'address'] = df.loc[46, 'address'] + ' ' + df.loc[46, 'TEL']

# TEL에 NULL 문자열 주기
df.loc[46, 'TEL'] = 'NULL'

# 최종 확인
df.loc[46:46]

,francise_name,address,TEL
46,용인구성점,"경기 용인시 기흥구 용구대로 2398 (마북동, 연원마을벽산아파트) 상가동 212호...",NULL


### CSV로 저장하기

In [229]:
#csv로 저장하기
df.to_csv('compose.csv',index=True, encoding='cp949')

pd.read_csv('compose.csv', encoding= 'cp949')


,Unnamed: 0,francise_name,address,TEL
0,0,일산만자고개점,경기 고양시 일산서구 덕이로 225-3 2층 203호,031-911-0667
1,1,경기광주밀목점,경기 광주시 중앙로 327,NaN
2,2,수원시청역점,경기 수원시 팔달구 효원로 278 파비오더씨타 109호,NaN
3,3,인천청학용담공원점,인천 연수구 용담로 44 102호,032-221-0032
4,4,양산경동스마트점,경남 양산시 신덕계로 34 상가동 102호,055-383-8382
...,...,...,...,...
2524,2524,정관1호점,부산 기장군 정관읍 정관로 385 (모전리),051-817-4668
2525,2525,부산교대점,부산 연제구 교대로 18 (거제동),051-506-0355
2526,2526,컴포즈커피 동서대점,부산 사상구 가야대로366번길 13 (주례동),051-315-8650
2527,2527,동아대점,"부산 사하구 낙동대로516번길 48 (하단동, 하단에덴아파트)",051-294-2638
